In [1]:
from helpers import *

Loading environment football failed: No module named 'gfootball'


In [2]:
from agents.rule_based_agent import RuleBasedAgent
from agents.greedy_agent import GreedyAgent

In [3]:
env = make("hungry_geese", debug=True)
config = env.configuration

In [4]:
step_reward = 1
winning_reward = 10
losing_reward = -10
discount = 0.8


def discounted(discount_factor, nb_steps, step_reward):
    discounted_reward = 0
    for _ in range(int(nb_steps)):
        discounted_reward = step_reward + discount_factor*discounted_reward
    return discounted_reward

step_200_reward = lambda my_goose, longuest_opponent: winning_reward if my_goose > longuest_opponent else 3*losing_reward
win_game_reward = lambda step, my_goose, longuest_opponent: winning_reward + discounted(discount, 200-step, step_reward) #max((200-step), winning_reward)

discount = 0.85

nb_opponents = 3

steps_per_ep = 200
num_episodes = 1000
nb_files = 100

In [5]:
for it in range(23, nb_files):
    print(f'starting iteration {it}')
    name = f'it_{it}_v2.pkl'
    episodes = []
    for ep in range(num_episodes):
        print('episode number: ', ep)
        steps = []
        my_agent = RuleBasedAgent()
        agents =  [my_agent] + [(RuleBasedAgent() if np.random.rand()<1 else GreedyAgent()) for _ in range(nb_opponents)]
        state_dict = env.reset(num_agents=nb_opponents + 1)[0]
        observation = state_dict['observation']
        my_goose_ind = observation['index']

        reward = state_dict['reward']
        action = state_dict['action']



        done = False
        for step in range(1, steps_per_ep):
            actions = []

            for i, agent in enumerate(agents):
                obs = deepcopy(observation)
                obs['index'] = i
                action = agent(obs, config)
                actions.append(action)

            state_dict = env.step(actions)[0]
            observation = state_dict['observation']
            my_goose_ind = observation['index']

            my_goose_length = len(observation['geese'][my_goose_ind])

            longuest_opponent=0
            for i, goose in enumerate(observation.geese):
                if i != my_goose_ind:
                    opponent_length = len(goose)
                    if opponent_length > longuest_opponent:
                        longuest_opponent = opponent_length

            #new_state, _, _ = agent.getStateSpace(observation, config)

            #reward = state_dict['reward']
            action = state_dict['action']
            status = state_dict['status']

            if status != "ACTIVE":
                done = True

            # Check if my goose died
            if my_goose_length == 0:
                done = True
                reward = losing_reward
            elif (step+1) == steps_per_ep:
                reward = step_200_reward(my_goose_length, longuest_opponent)
                done = True
            elif status != "ACTIVE":
                reward = win_game_reward(step, my_goose_length, longuest_opponent)
            else:
                reward = step_reward

            steps.append({'cur_state': my_agent.stateSpace,
                                    'action': action,
                                    'reward': reward,
                                    'new_state': '',#new_state,
                                    'status': status,
                                    'done': done})
            if done:
#                 print('Done, Step: ', step+1)
#                 print('status, ', status)
                break

            if step%50 == 0:
                pass
                #print(f'We survived {step+1} steps')
        episodes.append(steps)
    process(discount, episodes)
    train_data = training_data(episodes)
    with open(f'/home/charles/PycharmProjects/HungryGeese/data/{name}', 'wb') as f:
        pickle.dump(train_data, f)

starting iteration 0
episode number:  0
Goose Collision: SOUTH
Goose Collision: NORTH
episode number:  1
Goose Collision: NORTH
episode number:  2
Goose Collision: EAST
Goose Collision: NORTH
Goose Collision: SOUTH
episode number:  3
Body Hit: (2, <Action.NORTH: 1>, 49, [60, 59, 48, 49])
Body Hit: (0, <Action.SOUTH: 3>, 60, [49, 38, 27, 28, 39, 50, 61, 60, 59, 48, 37])
episode number:  4
Goose Collision: WEST
Goose Collision: NORTH
Body Hit: (0, <Action.EAST: 2>, 48, [47, 46, 45, 44, 33, 34, 35, 36, 37, 48, 59, 58])
episode number:  5
Body Hit: (0, <Action.WEST: 4>, 22, [23, 34, 45, 56, 55, 44, 33, 22, 32, 21, 20, 31])
episode number:  6
Body Hit: (1, <Action.SOUTH: 3>, 15, [4, 70, 59, 48, 37, 26, 15, 16])
Body Hit: (3, <Action.WEST: 4>, 5, [6, 7, 8, 74, 73, 72, 71, 5, 16, 17, 18, 19, 20])
Body Hit: (2, <Action.NORTH: 1>, 21, [32, 43, 42, 31, 20, 21, 11, 22, 23, 34, 45])
episode number:  7
Body Hit: (3, <Action.SOUTH: 3>, 41, [30, 29, 40, 41, 42, 31, 32, 43, 33])
episode number:  8
Goo